In [1]:
!pip install git+https://github.com/OlivierBinette/StringCompare.git@release

  Cloning https://github.com/OlivierBinette/StringCompare.git (to revision release) to /tmp/pip-req-build-hoeq7xfk
  Running command git clone -q https://github.com/OlivierBinette/StringCompare.git /tmp/pip-req-build-hoeq7xfk
  Running command git checkout -b release --track origin/release
  Switched to a new branch 'release'
  Branch 'release' set up to track remote branch 'release' from 'origin'.
  Resolved https://github.com/OlivierBinette/StringCompare.git to commit e3c3fead4b62dfa68248c0717c9c4c0f2a4e6f10
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
import stringcompare
import pandas as pd
import numpy as np
import wget
import zipfile
import os

if not os.path.isfile("rawinventor.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/rawinventor.tsv.zip")
    with zipfile.ZipFile("rawinventor.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("rawinventor.tsv.zip")

In [3]:
rawinventor = pd.read_csv("rawinventor.tsv", sep="\t", dtype=str, usecols=["patent_id", "sequence", "inventor_id", "name_first", "name_last"])

In [11]:
rawinventor

,patent_id,inventor_id,name_first,name_last,sequence
0,6584128,fl:ri_ln:kroeger-1,Richard,Kroeger,0
1,4789863,fl:th_ln:bush-1,Thomas A.,Bush,0
2,11161990,fl:ma_ln:boudreaux-4,Matthew F.,Boudreaux,1
3,6795487,fl:ge_ln:whitworth-1,Gerald,Whitworth,1
4,D474886,fl:th_ln:fleming-3,Thomas W.,Fleming,0
...,...,...,...,...,...
19632552,10261193,fl:ha_ln:doppke-1,Harald,Doppke,2
19632553,5441952,fl:da_ln:claremon-1,David A.,Claremon,0
19632554,4828281,zzzzquzhq278y507kbtfikz9b,David K.,Sanchas,0
19632555,10541390,fl:hi_ln:yonemura-3,Hitoshi,Yonemura,0


In [4]:
comparator = stringcompare.Levenshtein()

def get_sequence(patent_id, name_first, name_last):
    dat = rawinventor[rawinventor.patent_id == patent_id]
    distances = comparator.pairwise([name_last.lower()], dat.name_last.str.lower().values)[0]
    if sum(distances == 0) == 1 or sum(distances < 0.2) == 1: 
        return dat.sequence.values[np.argmin(distances)]
    else:
        return "NaN"

In [5]:
lai_benchmark = pd.read_excel("benchmark.xlsx", dtype=str).iloc[3:, :]

In [6]:
lai_benchmark["sequence"] = lai_benchmark.apply(lambda x: get_sequence(x.Patent, x.Firstname, x.Lastname), axis=1)

In [8]:
lai_benchmark.to_csv("benchmark_auto_sequence.csv")